<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Guilherme_Pereira/Aula_9_Guilherme_Pereira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Guilherme Pereira'
print(f'Meu nome é {nome}')

Meu nome é Guilherme Pereira


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Thu Jun  2 00:27:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List
from tqdm.notebook import tqdm


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int, iterador: bool):
        # Escreva aqui seu código.

        if iterador:
            self.ids = []
            for i in tqdm(range(len(texts))):
                self.ids.append(tokenizer.batch_encode_plus([texts[i]], return_tensors=None, add_special_tokens=False).input_ids[0])                
            # for i in range(len(self.ids)):
            #     self.ids[i] = self.ids[i][0]
        else:
            self.ids = tokenizer.batch_encode_plus(texts, return_tensors=None, add_special_tokens=False).input_ids

        self.iterador       = iterador
        self.dict_ids       = {}
        self.tokenizer      = tokenizer
        self.max_seq_length = max_seq_length

        total = 0

        for i in tqdm(range(len(self.ids))):
            itera = len(self.ids[i])//(self.max_seq_length-1) if len(self.ids[i])%(self.max_seq_length-1) == 0 else (len(self.ids[i])//(self.max_seq_length-1))+1
            # print(itera)
            for j in range(itera):
                self.dict_ids[total + j] = [i,j]

            total += itera

        
    def text_to_token(self, idx):

        # token_ids = tokenize(text=self.texts[idx], tokenizer=self.tokenizer)

        max0 = self.max_seq_length
        max1 = self.max_seq_length-1

        len_a = len( self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max1])
        len_b = len( self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max0])
        # print(len_a, len_b)

        limite_a = self.max_seq_length - len_a - 1
        limite_b = self.max_seq_length - len_b
        # print(limite_a, limite_b)

        p_a = self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max1]
        p_b = self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : self.dict_ids[idx][1]*max1 + max0]

        a = [self.tokenizer.cls_token_id] + p_a + [self.tokenizer.pad_token_id]*max(0, limite_a)
        b = p_b + [self.tokenizer.pad_token_id]*max(0, limite_b)

        # print(self.dict_ids[idx])
        # print(token_ids)
        # print(a)
        # print(b)

        return (torch.tensor(a), torch.tensor(b))
                
    def __len__(self):      
        return len(self.dict_ids)        

    def __getitem__(self, idx):
        return self.text_to_token(idx)

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9, iterador=True)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [ ]:
%%time
text = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=text, tokenizer=tokenizer, max_seq_length=3, iterador=True)
dummy_loader = DataLoader(dummy_dataset, batch_size=8, shuffle=False)
print(next(iter(dummy_loader)))
# print(dummy_dataset.ids)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

[tensor([[  101,  3396, 10303],
        [  101,   125, 13239],
        [  101,  1660,  5971],
        [  101,   785,   125],
        [  101,  1847, 13779],
        [  101, 15616,     0]]), tensor([[ 3396, 10303,   125],
        [  125, 13239,     0],
        [ 1660,  5971,   785],
        [  785,   125,  1847],
        [ 1847, 13779, 15616],
        [15616,     0,     0]])]
CPU times: user 90.6 ms, sys: 1.57 ms, total: 92.1 ms
Wall time: 201 ms


In [ ]:
%%time
text = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=text, tokenizer=tokenizer, max_seq_length=3, iterador=False)
dummy_loader = DataLoader(dummy_dataset, batch_size=8, shuffle=False)
print(next(iter(dummy_loader)))
# print(dummy_dataset.ids)

  0%|          | 0/2 [00:00<?, ?it/s]

[tensor([[  101,  3396, 10303],
        [  101,   125, 13239],
        [  101,  1660,  5971],
        [  101,   785,   125],
        [  101,  1847, 13779],
        [  101, 15616,     0]]), tensor([[ 3396, 10303,   125],
        [  125, 13239,     0],
        [ 1660,  5971,   785],
        [  785,   125,  1847],
        [ 1847, 13779, 15616],
        [15616,     0,     0]])]
CPU times: user 48.2 ms, sys: 1.29 ms, total: 49.5 ms
Wall time: 116 ms


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-02 00:27:49--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.153.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   138MB/s    in 9.9s    

2022-06-02 00:27:59 (118 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [ ]:
# Load datasets
max_seq_length = 9

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

train_examples = int(len(texts)*0.70*0.5)
valid_examples = int(len(texts)*0.15*0.5)
test_examples  = int(len(texts)*0.15*0.5)

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
print(f'Train Examples: {train_examples}')
print(f'Valid Examples: {valid_examples}')
print(f'Test Examples:  {test_examples}')
texts = texts[:max_lines]  

Read 250000 lines.
Truncating to 125000 lines.
Train Examples: 87500
Valid Examples: 18750
Test Examples:  18750


In [ ]:
print('\nLoadind Validation Texts ...')
valid_dataset    = MyDataset(texts=texts[-(valid_examples + test_examples):-test_examples], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)


Loadind Validation Texts ...


  0%|          | 0/18750 [00:00<?, ?it/s]

  0%|          | 0/18750 [00:00<?, ?it/s]

In [ ]:
print('\nLoadind Testing Texts ...')
test_dataset     = MyDataset(texts=texts[-test_examples:], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)


Loadind Testing Texts ...


  0%|          | 0/18750 [00:00<?, ?it/s]

  0%|          | 0/18750 [00:00<?, ?it/s]

In [ ]:
print('\nLoadind Training Texts ...')
training_dataset = MyDataset(texts=texts[:-(valid_examples + test_examples)], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)


Loadind Training Texts ...


  0%|          | 0/87500 [00:00<?, ?it/s]

  0%|          | 0/87500 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 12246016
valid examples: 2701364
test examples: 2568057


In [35]:
# Define model of double convolution
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, dim, max_seq_length, n_heads, pad_token_id):
        super(SelfAttention, self).__init__()

        self.dim            = dim
        self.max_seq_length = max_seq_length
        self.n_heads        = n_heads
        self.pad_token_id   = pad_token_id
        self.D_k            = dim//n_heads
        
        self.W_q = torch.nn.Linear(self.dim, self.dim, bias=False) # D, D
        self.W_k = torch.nn.Linear(self.dim, self.dim, bias=False) # D, D
        self.W_v = torch.nn.Linear(self.dim, self.dim, bias=False) # D, D
        self.W_o = torch.nn.Linear(self.dim, self.dim, bias=False) # D, D

        self.layer_norm = torch.nn.LayerNorm([self.max_seq_length, self.dim], eps=1e-6) # L, D


    def attention(self, Q, K, V, inputs):   
        
        scores = torch.matmul(Q, K.transpose(-1, -2))/math.sqrt(self.D_k) # B, HEADS, D, D
        mask   = inputs != self.pad_token_id                              # B, D

        mask_expanded = mask[:, None, None, :].expand_as(scores)  # B, HEADS, D, D
        scores.masked_fill_(~mask_expanded, float('-inf'))        # B, HEADS, D, D
        probs = F.softmax(scores, dim=-1)                         # B, HEADS, D, D

        E = torch.matmul(probs, V)                                    # B, HEADS, L, D//HEADS
        E = E.transpose(1,2).contiguous()                             # B, L, HEADS, D//HEADS
        E = E.reshape(inputs.shape[0], self.max_seq_length, self.dim) # B, L, D
        E = self.W_o(E)                                               # B, L, D

        return E
        
    def forward(self, x, inputs):

        q = self.W_q(x).reshape(inputs.shape[0],self.max_seq_length,self.n_heads,self.D_k).transpose(1,2) # B, HEADS, L, D//HEADS
        k = self.W_k(x).reshape(inputs.shape[0],self.max_seq_length,self.n_heads,self.D_k).transpose(1,2) # B, HEADS, L, D//HEADS
        v = self.W_v(x).reshape(inputs.shape[0],self.max_seq_length,self.n_heads,self.D_k).transpose(1,2) # B, HEADS, L, D//HEADS

        x = self.attention(q, k, v, inputs) # B, L, D
        x = self.W_o(x)                     # B, L, D

        return self.layer_norm(x)           # B, L, D

In [36]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.

        super().__init__()

        self.vocab_size     = vocab_size
        self.max_seq_length = max_seq_length
        self.dim            = dim
        self.n_layers       = n_layers
        self.pad_token_id   = pad_token_id
        self.n_heads        = 2
        

        self.embedding_layer       = torch.nn.Embedding(self.vocab_size, self.dim)
        self.positional_embeddings = torch.nn.Linear(self.dim, self.max_seq_length, bias=False)

        self.att_layers = nn.ModuleList()

        for i in range(self.n_layers):
            self.att_layers.append(SelfAttention(self.dim, self.max_seq_length, self.n_heads, self.pad_token_id))

        self.feed_forward = torch.nn.Sequential(nn.Linear(self.max_seq_length*self.dim, 8*self.dim),
                                                torch.nn.ReLU(),
                                                nn.Dropout(p=0.2),

                                                nn.Linear(8*self.dim, 4*self.dim),
                                                torch.nn.ReLU(),
                                                nn.Dropout(p=0.2),

                                                nn.Linear(4*self.dim, 2*self.dim),
                                                torch.nn.ReLU(),
                                                nn.Dropout(p=0.2),

                                                nn.Linear(2*self.dim, self.max_seq_length*vocab_size))# (2 layers with a ReLU in-between) 

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.

        x = self.embedding_layer(inputs) + self.positional_embeddings.weight # B, L, D

        for att in self.att_layers:
            x_att = att(x, inputs)     
            x = x + x_att  # B, L, D
                  
        x = self.feed_forward(x.reshape(len(inputs),-1))  # B, L*D

        logits = x.reshape(x.shape[0], self.max_seq_length, self.vocab_size)  # B, L, V
        
        return logits
    
    

## Teste o modelo com um exemplo

In [37]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=256,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset, batch_size=5)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([5, 9])
sample_output.shape: torch.Size([5, 9, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 36992946


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30051
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
path = "/content/drive/MyDrive/Guilherme/MODEL_NLP/"

Mounted at /content/drive


In [ ]:
from tqdm.notebook import tqdm

max_examples = 60_000_000
eval_every_steps = 10_000
lr = 4.5e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=2000, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=2000)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
ver = 1
print('====================== TRAINING MODEL ======================\n')
with tqdm(total=eval_every_steps) as pbar:
    print('\n====================== VALIDING MODEL ======================\n')
    while n_examples < max_examples:    
        for train_input_ids, train_target_ids in train_loader:
            loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
            train_losses.append(loss)
            # print(step)
            
            if step % eval_every_steps == 0:
                pbar.reset(total=eval_every_steps)
                train_ppl = np.exp(np.average(train_losses))

                with torch.no_grad():
                    valid_list = []
                    
                    for val_input_ids, val_target_ids in tqdm(validation_loader):
                        valid_list.append(validation_step(val_input_ids.to(device), val_target_ids.to(device)))
                    valid_ppl = np.exp(np.average(valid_list))

                print(f'\n{step:7d} steps; {(n_examples/max_examples)*100:8.2f} % completed; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
                train_losses = []

            if n_examples % 20_000_000 == 0:
                torch.save(model.state_dict(), path + 'save_v' + str(ver) + '.pth')
                ver += 1

            n_examples += len(train_input_ids)  # Increment of batch size
            step += 1
            pbar.update(1) 
            if n_examples >= max_examples:
                break

====================== TRAINING MODEL ======================



  0%|          | 0/10000 [00:00<?, ?it/s]


====================== VALIDING MODEL ======================



  0%|          | 0/1351 [00:00<?, ?it/s]


      0 steps;     0.00 % completed; 0 examples so far; train ppl: 29959.68, valid ppl: 29651.58


  0%|          | 0/1351 [00:00<?, ?it/s]


  10000 steps;    33.33 % completed; 20000000 examples so far; train ppl: 90.53, valid ppl: 11.90


  0%|          | 0/1351 [00:00<?, ?it/s]


  20000 steps;    66.67 % completed; 40000000 examples so far; train ppl: 29.39, valid ppl: 6.65


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

ver = 3
model.load_state_dict(torch.load(path + 'save_v' + str(ver) + '.pth'))

<All keys matched successfully>

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_list = []
    for test_input_ids, test_target_ids in tqdm(test_loader):
        test_list.append(validation_step(test_input_ids.to(device), test_target_ids.to(device)))
    test_ppl = np.exp(np.average(test_list))

print(f'test perplexity: {test_ppl}')

  0%|          | 0/40126 [00:00<?, ?it/s]

test perplexity: 6.535189799322884


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz o
Eu gosto de comer pizza pois me faz o que
Eu gosto de comer pizza pois me faz o que o
Eu gosto de comer pizza pois me faz o que o que
Eu gosto de comer pizza pois me faz o que o que não
Eu gosto de comer pizza pois me faz o que o que não não
Eu gosto de comer pizza pois me faz o que o que não não a
Eu gosto de comer pizza pois me faz o que o que não não a ser
Eu gosto de comer pizza pois me faz o que o que não não a ser que
Eu gosto de comer pizza pois me faz o que o que não não a ser que é
Eu gosto de comer pizza pois me faz o que o que não não a ser que é que
Eu gosto de comer pizza pois me faz o que o que não não a ser que é que não
Eu gosto de comer pizza pois me faz o que o que não não a ser que é que não não
Eu gosto de comer pizza pois me faz o que o que não não a ser que é que não não não
Eu gosto de comer pizza pois me faz o que o que não não a ser que é que não não não não
Eu gosto de comer pizza pois me faz o que o que não não a ser que 

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.